# Importing Necessary Libraries

In [2]:
# Libraries from PyTorch
import torch
from torch import nn
from torchvision import datasets,transforms,models
from torch.utils.data import Dataset,DataLoader

# Libraries for data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Libraries that makes life easier
from timeit import default_timer as timer
from pathlib import Path
from tqdm.auto import tqdm
import opendatasets as od
import cv2 as cv
from collections import OrderedDict
import os
from torchinfo import summary

# Libraries from Scikit-learn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

###### Execute this next line of code if you are accessing this notebook for the first time

In [3]:
# %run Curating_dataset_from_FER2013

# Loading the curated dataset

In [5]:
# Load and preprocess data
df_train = pd.read_csv("./FER_curated_dataset/train.csv")
df_train

,Unnamed: 0,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,48x40,48x41,48x42,48x43,48x44,48x45,48x46,48x47,48x48,label
0,0,255,255,255,156,213,255,255,128,75,...,255,255,255,255,255,255,124,205,205,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,255,255,255,255,255,255,255,255,255,...,111,147,176,203,251,246,234,188,76,0
3,3,0,0,0,0,0,0,0,0,8,...,255,255,255,255,255,255,99,25,35,0
4,4,230,255,251,235,230,255,205,255,211,...,148,77,82,118,102,109,104,114,154,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,4965,186,197,186,191,215,255,255,255,237,...,157,255,255,255,255,255,255,255,255,5
4966,4966,146,169,187,209,232,241,237,231,228,...,219,229,255,255,255,255,255,255,255,5
4967,4967,255,255,79,255,255,255,255,255,255,...,197,218,224,225,219,221,229,233,242,5
4968,4968,226,222,194,223,243,235,248,241,143,...,255,255,255,255,255,255,255,255,255,5


In [6]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_train

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,48x40,48x41,48x42,48x43,48x44,48x45,48x46,48x47,48x48,label
0,255,255,255,156,213,255,255,128,75,167,...,255,255,255,255,255,255,124,205,205,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,255,255,255,255,255,255,255,255,255,255,...,111,147,176,203,251,246,234,188,76,0
3,0,0,0,0,0,0,0,0,8,3,...,255,255,255,255,255,255,99,25,35,0
4,230,255,251,235,230,255,205,255,211,255,...,148,77,82,118,102,109,104,114,154,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,186,197,186,191,215,255,255,255,237,254,...,157,255,255,255,255,255,255,255,255,5
4966,146,169,187,209,232,241,237,231,228,237,...,219,229,255,255,255,255,255,255,255,5
4967,255,255,79,255,255,255,255,255,255,165,...,197,218,224,225,219,221,229,233,242,5
4968,226,222,194,223,243,235,248,241,143,116,...,255,255,255,255,255,255,255,255,255,5


In [7]:
df_test = pd.read_csv("./FER_curated_dataset/test.csv")
df_test

,Unnamed: 0,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,48x40,48x41,48x42,48x43,48x44,48x45,48x46,48x47,48x48,label
0,0,0,1,2,3,4,5,6,7,8,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,7
1,1,255,255,255,156,213,255,255,128,75,...,255,255,255,255,255,255,124,205,205,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,255,255,255,255,255,255,255,255,255,...,210,206,198,202,215,227,231,233,225,0
4,4,255,255,255,255,255,255,255,255,255,...,255,253,243,255,254,242,242,239,233,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28774,28774,146,169,187,209,232,241,237,231,228,...,219,229,255,255,255,255,255,255,255,5
28775,28775,255,255,79,255,255,255,255,255,255,...,197,218,224,225,219,221,229,233,242,5
28776,28776,255,255,255,250,255,255,255,255,248,...,255,255,255,255,255,255,226,200,187,5
28777,28777,226,222,194,223,243,235,248,241,143,...,255,255,255,255,255,255,255,255,255,5


In [8]:
df_test = df_test.drop([0])
df_test = df_test.drop(['Unnamed: 0'],axis=1)
df_test

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,48x40,48x41,48x42,48x43,48x44,48x45,48x46,48x47,48x48,label
1,255,255,255,156,213,255,255,128,75,167,...,255,255,255,255,255,255,124,205,205,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,255,255,255,255,255,255,255,255,255,211,...,210,206,198,202,215,227,231,233,225,0
4,255,255,255,255,255,255,255,255,255,255,...,255,253,243,255,254,242,242,239,233,0
5,177,182,161,183,230,255,252,197,150,141,...,255,255,217,251,222,83,107,255,248,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28774,146,169,187,209,232,241,237,231,228,237,...,219,229,255,255,255,255,255,255,255,5
28775,255,255,79,255,255,255,255,255,255,165,...,197,218,224,225,219,221,229,233,242,5
28776,255,255,255,250,255,255,255,255,248,238,...,255,255,255,255,255,255,226,200,187,5
28777,226,222,194,223,243,235,248,241,143,116,...,255,255,255,255,255,255,255,255,255,5


# Accessing the images and the labels

In [9]:
# From train.csv
x_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values

# From test.csv
x_test = df_test.iloc[:,:-1].values
y_test = df_test.iloc[:,-1].values

In [10]:
# Verify feature sizes
print("Feature shape before reshaping:", x_train.shape)

# Ensure the correct feature size
if x_train.shape[1] != 48 * 48:
    raise ValueError(f"Each feature should have 2304 elements, but got {x.shape[1]}")

Feature shape before reshaping: (4970, 2304)


# Defining a Custom Dataset

In [11]:
class class_Dataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features.astype(np.float32)  # Ensure features are float32
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        
        # Reshape the feature into the original image shape (48, 48)
        feature = feature.reshape(48, 48)
        
        if self.transform:
            feature = self.transform(feature)
        
        return feature, label

# Applying transformations to the dataset

In [12]:
# Define transformations for the images
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors and scale to [0, 1]
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize images
])

# Create instances of the custom dataset
train_dataset = class_Dataset(x_train, y_train, transform=transform)
test_dataset = class_Dataset(x_test, y_test, transform=transform)
len(train_dataset),len(train_dataset)

(4970, 4970)

# Creating DataLoaders

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1d3f1011a10>,
 <torch.utils.data.dataloader.DataLoader at 0x1d3ef06da10>)

# Setting-up device agnostic code

In [15]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')
# device = torch.device('cpu')
device

device(type='cuda')

# Building the Model

In [16]:
class DetectionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,padding=1)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,padding=1)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216,1024),
            nn.ReLU(),
            nn.Linear(1024,1024),
            nn.ReLU(),
            nn.Linear(1024,6)
        )
    
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.classifier(x)
        
        return x
    
model = DetectionModel().to(device)
model

DetectionModel(
  (block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9216, out_features=1024, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10

In [17]:
summary(model,input_shape=(1,1,48,48))

Layer (type:depth-idx)                   Param #
DetectionModel                           --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       320
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       18,496
│    └─ReLU: 2-4                         --
│    └─MaxPool2d: 2-5                    --
├─Sequential: 1-2                        --
│    └─Conv2d: 2-6                       73,856
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       147,584
│    └─ReLU: 2-9                         --
│    └─MaxPool2d: 2-10                   --
├─Sequential: 1-3                        --
│    └─Conv2d: 2-11                      295,168
│    └─ReLU: 2-12                        --
│    └─Conv2d: 2-13                      590,080
│    └─ReLU: 2-14                        --
├─Sequential: 1-4                        --
│    └─Flatten: 2-15                     --
│    └─Linear: 2-16                      9,438,

# Setting-up Loss Function and optimizer

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.00001)

# Code for stopping the training loop early

In [19]:
# Early stopping class definition (if not already defined)
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        torch.save(model.state_dict(), 'checkpoint.pth')
        if self.verbose:
            print(f'Validation loss decreased. Saving model ...')

# Initialize early stopping
early_stopping = EarlyStopping(patience=10, verbose=True)

In [20]:
train_images,train_labels = next(iter(train_dataloader))
print(f"Shape of image: {train_images.shape} | Shape of label: {train_labels.shape}")

Shape of image: torch.Size([64, 1, 48, 48]) | Shape of label: torch.Size([64])


# Making the Training Loop

In [21]:
# Training loop
num_epochs = 50
trainloss, testloss = [], []
trainaccuracy, testaccuracy = [], []

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    
    for train_images, train_labels in train_dataloader:
        train_images, train_labels = train_images.to(device), train_labels.to(device)
        optimizer.zero_grad()
        outputs = model(train_images)
        train_loss = loss_fn(outputs, train_labels)
        train_loss.backward()
        optimizer.step()
        
        _, train_pred = torch.max(outputs.data, dim=1)
        correct_train += (train_pred == train_labels).sum().item()
        running_train_loss += train_loss.item()
    
    avg_train_loss = running_train_loss / len(train_dataloader)
    train_accuracy = 100 * correct_train / len(train_dataloader.dataset)
    
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    
    with torch.no_grad():
        for test_images, test_labels in test_dataloader:
            test_images, test_labels = test_images.to(device), test_labels.to(device)
            test_outputs = model(test_images)
            test_loss = loss_fn(test_outputs, test_labels)
            _, test_pred = torch.max(test_outputs.data, 1)
            correct_test += (test_pred == test_labels).sum().item()
            running_test_loss += test_loss.item()
    
    avg_test_loss = running_test_loss / len(test_dataloader)
    test_accuracy = 100 * correct_test / len(test_dataloader.dataset)
    
    trainloss.append(avg_train_loss)
    testloss.append(avg_test_loss)
    trainaccuracy.append(train_accuracy)
    testaccuracy.append(test_accuracy)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | Test Loss: {avg_test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%')
    
    early_stopping(avg_test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Load the last checkpoint with the best model
model.load_state_dict(torch.load('checkpoint.pth'))


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50] | Train Loss: 1.7156 | Train Accuracy: 27.46% | Test Loss: 1.8273 | Test Accuracy: 18.35%
Validation loss decreased. Saving model ...
Epoch [2/50] | Train Loss: 1.3715 | Train Accuracy: 48.57% | Test Loss: 2.2562 | Test Accuracy: 19.50%
EarlyStopping counter: 1 out of 10
Epoch [3/50] | Train Loss: 1.0288 | Train Accuracy: 60.97% | Test Loss: 2.8560 | Test Accuracy: 19.81%
EarlyStopping counter: 2 out of 10
Epoch [4/50] | Train Loss: 0.8963 | Train Accuracy: 65.01% | Test Loss: 3.0766 | Test Accuracy: 20.84%
EarlyStopping counter: 3 out of 10
Epoch [5/50] | Train Loss: 0.7889 | Train Accuracy: 68.95% | Test Loss: 3.4348 | Test Accuracy: 20.95%
EarlyStopping counter: 4 out of 10
Epoch [6/50] | Train Loss: 0.7395 | Train Accuracy: 70.76% | Test Loss: 3.6051 | Test Accuracy: 21.80%
EarlyStopping counter: 5 out of 10
Epoch [7/50] | Train Loss: 0.7012 | Train Accuracy: 72.80% | Test Loss: 3.7660 | Test Accuracy: 21.36%
EarlyStopping counter: 6 out of 10
Epoch [8/50] | Train Loss

<All keys matched successfully>